In [2]:
import importlib
import sys,os,math,time

gostNetsFolder = os.path.dirname(os.getcwd())
sys.path.insert(0, gostNetsFolder)
from GOSTNets import GOSTnet as gn
from GOSTNets import OSMParser
importlib.reload(gn)

import networkx as nx
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
import rasterio

from shapely.geometry import Point
from shapely.ops import nearest_points

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


In [60]:
inputPBF = r"Q:\AFRICA\MRT\INFRA\mauritania-latest_20190103_OSMLR12.osm.xml"
inputBorders = r"Q:\AFRICA\MRT\INFRA\MRT_Fake_Border.shp"
originFile = r"Q:\AFRICA\MRT\INFRA\Origins.shp"
destFile = r"Q:\AFRICA\MRT\INFRA\Destinations.shp"

,node_ID,lat,lon,id,geometry
0,4244699825,16.945679,-15.194188,4244699825,POINT (-15.1941884 16.9456794)
1,2241915521,16.449583,-15.646090,2241915521,POINT (-15.6460905 16.4495825)
2,4603254440,15.269953,-12.868975,4603254440,POINT (-12.8689754 15.2699527)
3,5297654164,26.998836,-8.079721,5297654164,POINT (-8.0797206 26.9988361)
4,4186327402,14.921902,-12.523779,4186327402,POINT (-12.5237794 14.9219024)


In [40]:
#Read in the network object and convert to GDF
importlib.reload(OSMParser)
importlib.reload(gn)

G = OSMParser.read_osm(inputPBF)
G = gn.convert_network_to_time(G, distance_tag = 'length', speed_dict = gn.speed_dict)
edgeDF = gn.edge_gdf_from_graph(G, xCol='lon', yCol='lat')
nodeDF = gn.node_gdf_from_graph(G, xCol='lon', yCol='lat')

inB = gpd.read_file(inputBorders)
inO = gpd.read_file(originFile)
inD = gpd.read_file(destFile)

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


In [54]:
#Identify all network edges that intersect the borders file
adjustments = {}
for idx, row in inB.iterrows():
    #Select the roads that intersect this border
    ### TODO - use a spatial index here, it is too slow
    intersections = edgeDF[edgeDF.intersects(row['geometry'])]
    for selIdx, selRow in intersections.iterrows():
        adjustments[selRow['id']] = row['id']

In [59]:
#Loop through edges in network and add time
G_adj = G.copy()
for u,v,data in G_adj.edges(data=True):
    if data['id'] in adjustments.keys():
        data['time'] = data['time'] + adjustments[data['id']]

In [82]:
# identify intersecting nodes for input and output nodes
importlib.reload(gn)

inNodes = []
destNodes = []
for idx, row in inO.iterrows():
    nPt = nodeDF.distance(row['geometry'])
    inNodes.append(nodeDF['node_ID'][nPt.idxmin()])    

destNodes = []
for idx, row in inD.iterrows():
    nPt = nodeDF.distance(row['geometry'])
    destNodes.append(nodeDF['node_ID'][nPt.idxmin()])    


peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


In [87]:
#Run OD matrix
undistrubed = gn.calculate_OD(G, inNodes, destNodes, -1)
distrubed = gn.calculate_OD(G_adj, inNodes, destNodes, -1)

In [91]:
distrubed

array([[ 1.39535773e+05,  6.78733044e+04,  4.26197620e+04,
         2.18072575e+05,  1.42461643e+05,  1.44873726e+05,
         1.34954985e+05],
       [ 1.49363498e+05,  1.77353410e+05,  2.01605392e+05,
         2.54374069e+04,  1.03282744e+05,  1.39911843e+05,
         1.69704770e+05],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00],
       [ 7.79393099e+04,  1.05929221e+05,  1.30181204e+05,
         4.65422604e+04,  3.18585559e+04,  6.84876549e+04,
         9.82805814e+04],
       [ 7.05549500e+04,  1.60438247e+04,  4.02958071e+04,
         1.43257082e+05,  7.09093568e+04,  7.33214398e+04,
         8.86583795e+04],
       [ 2.33595996e+04,  5.45071632e+04,  7.87591455e+04,
         1.01561047e+05,  2.37036660e+04,  2.56906134e+04,
         4.37008711e+04],
       [ 1.02147646e+04,  6.91749603e+04,  8.79349868e+04,
         1.32951229e+05,  5.50938481e+04,  5.72200441e+04,
         1.1337577

In [92]:
undistrubed

array([[ 1.33467236e+05,  6.25933044e+04,  4.26197620e+04,
         2.09792575e+05,  1.33821643e+05,  1.36233726e+05,
         1.34954985e+05],
       [ 1.49363498e+05,  1.74353410e+05,  1.93325392e+05,
         2.54374069e+04,  1.03282744e+05,  1.39911843e+05,
         1.69704770e+05],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00],
       [ 7.79393099e+04,  1.02929221e+05,  1.21901204e+05,
         4.65422604e+04,  3.18585559e+04,  6.84876549e+04,
         9.82805814e+04],
       [ 6.71949500e+04,  1.60438247e+04,  3.50158071e+04,
         1.40257082e+05,  6.75493568e+04,  6.99614398e+04,
         8.75362215e+04],
       [ 2.33595996e+04,  5.11471632e+04,  7.01191455e+04,
         1.01561047e+05,  2.37036660e+04,  2.56906134e+04,
         4.37008711e+04],
       [ 1.02147646e+04,  6.91749603e+04,  8.79349868e+04,
         1.32951229e+05,  5.50938481e+04,  5.72200441e+04,
         1.1337577